## SEIS 764-01 Assignment 8
**Jamie Boehme**

Import libraries for processing

In [60]:
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import utils
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt

Read data from google colab drive

In [61]:
from google.colab import drive
drive.mount('/content/drive')
data_path = "/content/drive/My Drive/Colab Notebooks/CNN_Flowers"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [62]:
batch_size = 10

data = ImageDataGenerator(rescale=1. / 255, 
                          validation_split=0.2)

training_data1 = data.flow_from_directory('/content/drive/My Drive/Colab Notebooks/CNN_Flowers', 
                                         target_size=(200, 200), shuffle=True, batch_size = batch_size, 
                                         class_mode='categorical', subset='training')

training_data_noshuffle1 = data.flow_from_directory('/content/drive/My Drive/Colab Notebooks/CNN_Flowers', 
                                         target_size=(200, 200), shuffle=False, batch_size = batch_size, 
                                         class_mode='categorical', subset='training')

test_data1 = data.flow_from_directory('/content/drive/My Drive/Colab Notebooks/CNN_Flowers', 
                                     target_size=(200, 200), batch_size = batch_size, shuffle=False,
                                     class_mode='categorical', subset='validation')

training_data2 = data.flow_from_directory('/content/drive/My Drive/Colab Notebooks/CNN_Flowers', 
                                         target_size=(200, 200), shuffle=True, batch_size = batch_size, 
                                         class_mode='categorical', subset='training')

training_data_noshuffle2 = data.flow_from_directory('/content/drive/My Drive/Colab Notebooks/CNN_Flowers', 
                                         target_size=(200, 200), shuffle=False, batch_size = batch_size, 
                                         class_mode='categorical', subset='training')

test_data2 = data.flow_from_directory('/content/drive/My Drive/Colab Notebooks/CNN_Flowers', 
                                     target_size=(200, 200), batch_size = batch_size, shuffle=False,
                                     class_mode='categorical', subset='validation')

numClasses = 2

Found 90 images belonging to 2 classes.
Found 90 images belonging to 2 classes.
Found 22 images belonging to 2 classes.
Found 90 images belonging to 2 classes.
Found 90 images belonging to 2 classes.
Found 22 images belonging to 2 classes.


Train initial pre-built model

In [63]:
Classifier1=Sequential()
Classifier1.add(Conv2D(64, kernel_size=(5, 5), input_shape=(200, 200, 3)))
Classifier1.add(BatchNormalization())
Classifier1.add(Activation('relu'))
Classifier1.add(MaxPooling2D(pool_size=(2,2)))
Classifier1.add(Dropout(0.2))
               
Classifier1.add(Conv2D(32, kernel_size=(3, 3)))
Classifier1.add(BatchNormalization())
Classifier1.add(Activation('relu'))
Classifier1.add(MaxPooling2D(pool_size=(2,2)))
Classifier1.add(Dropout(0.2))

Classifier1.add(Flatten())

Classifier1.add(Dense(64, activation='relu'))
Classifier1.add(Dense(32, activation='relu'))
Classifier1.add(Dense(16, activation='relu'))
Classifier1.add(Dense(8, activation='relu'))
Classifier1.add(Dense(numClasses, activation='softmax'))

In [64]:
MyEpochs = 30
opt = keras.optimizers.Adam(learning_rate=0.01)
#opt = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)

Classifier1.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt, 
              metrics=['accuracy']) 

Classifier1.fit(training_data1,
                        batch_size = 10,
                        epochs = MyEpochs,
                        validation_data=test_data1,
                        shuffle = 1,
                        verbose =0)

In [65]:
training_data1.reset()
test_data1.reset()

predicted_scores1 = Classifier1.predict(test_data1, verbose=1)
predicted_labels1 = predicted_scores1.argmax(axis=1) 

test_labels1 = test_data1.labels

acc_score_pre = accuracy_score(test_labels1, predicted_labels1)
CFM1 = confusion_matrix(test_labels1, predicted_labels1)

3/3 [==============================] - 1s 191ms/step


Train a second model, that produces better results

In [66]:
Classifier2=Sequential()
Classifier2.add(Conv2D(16, kernel_size=(5, 5), input_shape=(200, 200, 3)))
Classifier2.add(BatchNormalization())
Classifier2.add(Activation('relu'))
               
Classifier2=Sequential()
Classifier2.add(Conv2D(32, kernel_size=(3, 3)))
Classifier2.add(BatchNormalization())
Classifier2.add(Activation('relu'))
Classifier2.add(Dropout(0.1))

Classifier2.add(Conv2D(64, kernel_size=(3, 3)))
Classifier2.add(BatchNormalization())
Classifier2.add(Activation('relu'))
Classifier2.add(MaxPooling2D(pool_size=(2,2)))
Classifier2.add(Dropout(0.1))

Classifier2.add(Flatten())

Classifier2.add(Dense(64, activation='relu'))
Classifier2.add(Dense(40, activation='relu'))
Classifier2.add(Dense(32, activation='relu'))
Classifier2.add(Dense(16, activation='relu'))
Classifier2.add(Dense(8, activation='relu'))
Classifier2.add(Dense(numClasses, activation='softmax'))

In [ ]:
MyEpochs = 40
opt = SGD(learning_rate=0.01)

Classifier2.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=opt, 
              metrics=['accuracy']) 

Classifier2.fit(training_data2,
                        batch_size = 6,
                        epochs = MyEpochs,
                        validation_data=test_data2,
                        shuffle = 1,
                        verbose=1)

3. Print the configurations (i.e. architecture) of all the layers in your CNN

In [68]:
Classifier2.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_51 (Conv2D)          (None, None, None, 32)    896       
                                                                 
 batch_normalization_48 (Bat  (None, None, None, 32)   128       
 chNormalization)                                                
                                                                 
 activation_50 (Activation)  (None, None, None, 32)    0         
                                                                 
 dropout_42 (Dropout)        (None, None, None, 32)    0         
                                                                 
 conv2d_52 (Conv2D)          (None, None, None, 64)    18496     
                                                                 
 batch_normalization_49 (Bat  (None, None, None, 64)   256       
 chNormalization)                                    

1. What are the testing accuracies before and after you modify the program?

In [69]:
training_data2.reset()
test_data2.reset()

predicted_scores2 = Classifier2.predict(test_data2, verbose=1)
predicted_labels2 = predicted_scores2.argmax(axis=1) 

test_labels2 = test_data2.labels

acc_score_post = accuracy_score(test_labels2, predicted_labels2)
CFM2 = confusion_matrix(test_labels2, predicted_labels2)


print("\n", "Accuracy of Pre-Trained Model: " + str(format(acc_score_pre,'.3f')))
print("\n", "Accuracy of 2nd, Better Model: " + str(format(acc_score_post,'.3f')))

3/3 [==============================] - 1s 317ms/step

 Accuracy of Pre-Trained Model: 0.864

 Accuracy of 2nd, Better Model: 0.955


2. Print the confusion matrix of your classification result before and after you
modify the program. 

In [70]:
print("\n", "Pre-trained Confusion Matrix: \n", CFM1)
print("\n", "2nd, Better Model Confusion Matrix: \n", CFM2)

print("\n", "Pre-trained classification report: \n", classification_report(test_labels1, predicted_labels1))
print("\n", "2nd, Better Model classification report: \n", classification_report(test_labels2, predicted_labels2))


 Pre-trained Confusion Matrix: 
 [[ 9  2]
 [ 1 10]]

 2nd, Better Model Confusion Matrix: 
 [[10  1]
 [ 0 11]]

 Pre-trained classification report: 
               precision    recall  f1-score   support

           0       0.90      0.82      0.86        11
           1       0.83      0.91      0.87        11

    accuracy                           0.86        22
   macro avg       0.87      0.86      0.86        22
weighted avg       0.87      0.86      0.86        22


 2nd, Better Model classification report: 
               precision    recall  f1-score   support

           0       1.00      0.91      0.95        11
           1       0.92      1.00      0.96        11

    accuracy                           0.95        22
   macro avg       0.96      0.95      0.95        22
weighted avg       0.96      0.95      0.95        22



3. What changes did you make to the program for the differences you saw in
question 1? I made the following changes to the initial program:
- Started my program with less filters on the original images, and grew the level of filters rather than decreasing the original filters. I did this, so that the model conforms to best practices for AI, increasing filters and deacreasing the size of the image over time to abstract local features.
- Grew my filters more slowly than the original model decreased filters, to allow the program to gradually learn over time - and decreased my fully connected layers more slowly.
- Reduced drop-out, knowing there were already limited images to see if I could still receive good testing results
- Decreased the batch size in order to increase backpropogation, and therefore faster learning of the images over each epoch
- Had additional fully connected layers to slow drop-off at each fully connected layer
- Increased the number of epochs, as it appeared the learning had not yet stalled at 30 epochs

Things to note: This model seems very fragile. It seems that the accuracy and precision can vary widely based upon each runtime.